In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
IDS2017URL = "/gdrive/MyDrive/IDS2017/IDS2017.csv"

In [3]:
import torch
import pandas as pd
import tensorflow as tf
import numpy as np
import torch.nn as nn
import glob
import time
import os
from os.path import join
from numpy import vstack
from numpy import argmax
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch import Tensor
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Softmax
from torch.nn import Module
from torch.optim import SGD
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_
from torchvision import transforms

In [4]:
seed_split = 40

In [5]:
!pip install https://github.com/KevinMMendez/cimcb/archive/master.zip

     / 983kB 6.3MB/s
  Created wheel for cimcb: filename=cimcb-2.1.0-cp37-none-any.whl size=166774 sha256=66b70a89094b67e393e05d260e1d73b2433e0d24e07dad6665b792986aa78410
  Stored in directory: /tmp/pip-ephem-wheel-cache-50o9c7bs/wheels/6b/15/b6/0bf9d1e5bbcace7b87c78d8bef68ae197bc4679dfb373a8f01
Successfully built cimcb


In [6]:
import numpy as np
import pandas as pd
import cimcb as cb
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [7]:
data = pd.read_csv(IDS2017URL)
data.loc[data['Label'].isin([ 'Web Attack - Sql Injection','Web Attack - XSS', 'Web Attack - Brute Force']), 'Label'] = 'Web Attack'
#data = data[['Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Flow IAT Max', 'Bwd Packet Length Std', 'Fwd Packet Length Max', 'Flow Bytes/s', 'Total Length of Bwd Packets', 'Fwd Packet Length Mean', 'Flow Duration', 'Flow IAT Min', 'Total Length of Fwd Packets', 'Flow IAT Mean', 'Total Backward Packets', 'Bwd Packet Length Max', 'Flow Packets/s', 'Flow IAT Std', 'Fwd IAT Total', 'Bwd Packet Length Min', 'Fwd Packet Length Min', 'Label']]
data = data[['Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Label']]
data = data.replace([-np.inf, np.inf], np.nan)
data = data.dropna()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (84) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
data.replace(to_replace=0, value=0.00001, inplace=True)

In [9]:
num_records, num_features = data.shape

In [10]:
data.shape

(2830743, 3)

In [11]:
X = data.iloc[:,:-1].astype(float).apply(pd.to_numeric)

In [12]:
X

,Fwd Packet Length Mean,Fwd Packet Length Std
0,6.000000,0.00001
1,6.000000,0.00001
2,6.000000,0.00001
3,6.000000,0.00001
4,6.000000,0.00001
...,...,...
2830738,0.000010,0.00001
2830739,6.000000,0.00001
2830740,0.000010,0.00001
2830741,49.333333,85.44784


In [13]:
le = LabelEncoder()
le.fit(["BENIGN", "DoS Hulk", "PortScan", "DDoS", "DoS GoldenEye","FTP-Patator","SSH-Patator","DoS slowloris","DoS Slowhttptest","Web Attack","Bot","Infiltration","Heartbleed"])
Y = le.transform(data['Label'])

In [14]:
Class = data['Label']

In [15]:
XTrain, XTest, YTrain, YTest, ClassTrain, ClassTest = train_test_split(X,
                                                                       Y,
                                                                       Class,
                                                                       test_size=1/3,
                                                                       stratify=Y,
                                                                       random_state=seed_split)

In [16]:
XTrain

,Fwd Packet Length Mean,Fwd Packet Length Std
1781920,53.000000,0.000010
547912,11.222222,8.828615
1800450,12.815789,43.025985
2343779,26.000000,28.284271
1594355,40.000000,0.000010
...,...,...
2043289,2.000000,0.000010
1308697,53.000000,0.000010
141105,22.333333,16.010413
1405096,67.000000,0.000010


In [ ]:
XTrainLog = np.log(XTrain)                                          
XTrainScale, mu, sigma = cb.utils.scale(XTrainLog, method='auto', return_mu_sigma=True)              
XTrainKnn = cb.utils.knnimpute(XTrainScale, k=3)    

# Extract X Test Data
XTestLog = np.log(XTest)                                          
XTestScale = cb.utils.scale(XTestLog, method='auto', mu=mu, sigma=sigma)           
XTestKnn = cb.utils.knnimpute(XTestScale, k=3)